In [35]:
import pandas as pd
import numpy as np 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [ ]:

from matplotlib import rcParams
# setting figure size default
rcParams['figure.figsize'] = 30,20

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [28]:
try:
    df = pd.read_csv(r'C:\Users\Kazi\Udacity_2\Udacity\Data Scientist Nano\Term2\Project1\developer_survey_2019\survey_results_public.csv')
except:
    df = pd.read_csv(r'C:\Users\mkazi\Desktop\Kazi\Udacity\GoogleDrive\Udacity\Data Scientist Nano\Term2\Project1\developer_survey_2019\survey_results_public.csv')



lets hone down on data analysts and data scientists
1. which country pays the most to these professsionals
2. what languages do they prefer?
3. what's tehir job satiusfaction like?
4. what is teh ier experience like?
5. whiat country has the most data scientists, as a percent of total devs
6. what is ther education like


To do this, we needed to 
1. remove non-professional prgrammers
2. impute saalries
3. take median salary
4. remove low respondent countries
5. isolate datascientists and analysts
6. reclassify other professions


In [30]:
# Step 1 - all professionals

df1 = df[df.MainBranch.isin(["I am not primarily a developer, but I write code sometimes as part of my work",'I am a developer by profession'])]


In [ ]:
# step 2 - remove countries with les than 100 respondents

df2 = df1.groupby('Country').Respondent.nunique().reset_index()
Countries = list(set(df2[df2.Respondent >= 100 ].Country))
df3 = df1[df1.Country.isin(Countries)]

In [ ]:
# step 3 - separate data scientists and analysts from other professions
datapeople = ['Data scientist','Data or business analyst']
df3['Profession'] = np.where((df3['DevType'].str.contains('|'.join(datapeople))), 'DataPerson','Other')

In [ ]:
df3

# Quiestions 1 
 - which country payts the best data science salaries, and how does that copmpare to other professions


<br>
there appears to be a lot of missing values in the Salary depratment. We should fill these in before proceeding
<br>

well ot find out we must first impute salaries that are missing, and remove outliers. We can accmomplish both of these by imputing the median 
 
 - Our median salary is imoputed by looking at 3 factors"
  1. The country theyre in
  2. The experience they have in coding
  3. our classifier of Data epoeple and everyoine else. 

In [ ]:
# step 4 - Fill in missing salaries  

def medianSalary(country,yearscoding,profession):
    sal = df3[(df3.Country == country) & (df3.YearsCodePro == yearscoding) & (df3.Profession == profession)].ConvertedComp.median()
    return sal

In [ ]:
  
df4 = df3.copy()
df4['ImputedSalary'] = df4.ConvertedComp.combine_first(df4[(df4.ConvertedComp.isna())].apply(lambda x: medianSalary(x['Country'] , x['YearsCodePro'], x['Profession']), axis = 1 ))

In [ ]:
df4

In [ ]:
# step 5 - grpahicng

df4.groupby('Country').ImputedSalary.median().reset_index().sort_values('ImputedSalary', ascending = False)


In [ ]:
pvt = df4.pivot_table(values=['ImputedSalary'], 
                      index='Country',
                      columns='Profession',
                      aggfunc='median')

df = pvt.reindex(pvt['ImputedSalary'].sort_values(by='DataPerson', ascending=False).index)

# pvt = pvt.div(pvt.sum(1), axis=0)
# pvt.plot(kind='bar', stacked=True)
df.columns = df.columns.droplevel(0)
df

In [ ]:
df.plot(kind='bar', figsize = (30,15))

# Answer 1:
From the above we can see that Data Analysts and Scientists actually get more than other professions, on average. 

<br>
There are definitely some countries where this isn't true. but it does hold true for the top 5 countries. 


In [ ]:
s = df4['LanguageWorkedWith'].str.split(';').apply(pd.Series, 1).stack()

s.index = s.index.droplevel(-1) # to line up with df's index

s.name = 'Languages' # needs a name to join

del df4['LanguageWorkedWith']

# Question 2

- Is there a language preference amongst data scientists and data analysts?
- Waht are they working in?
- and also, what do they want to work in next?

In [ ]:


df8 = df4.join(s)

pvt = df8.pivot_table(values=['Respondent'], 
                      index='Languages',
                      columns='Profession',
                      aggfunc='sum')

pvt = pvt.div(pvt.sum(1), axis=0)
pvt.plot(kind='bar', stacked=True)

# Answer 2
Well we have a winner folks!
- R is definetyly the preferred tool analysts uyse over other tools. 
- Runners up were VBA, Scala, and Python

# Question 3
 - Lastly, are the data people have more educated than the other dev counterparts?

In [ ]:
pvt

In [ ]:
pvt2 = pvt.T
pvt2 = pvt2.reset_index()

In [ ]:
pvt3 = pvt2.copy()

In [ ]:
pvt3['ProfCat'] = np.where(pvt3.EdLevel.isin(['I never completed any formal education','Primary/elementary school','Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)']), 'School',
                              np.where(pvt3.EdLevel.isin(['Bachelor’s degree (BA, BS, B.Eng., etc.)']),'Bachelor''s Degree',
                                        np.where(pvt3.EdLevel.isin(['Master’s degree (MA, MS, M.Eng., MBA, etc.)']),'Master''s Degree',
                                                 np.where(pvt3.EdLevel.isin(['Professional degree (JD, MD, etc.)','Other doctoral degree (Ph.D, Ed.D., etc.)']),'Professional or Doctoral Degree','Other Education'))))
                                      
                                      

In [ ]:
pvt4 = pvt3.groupby('ProfCat').agg({'DataPerson':'sum', 'Other': 'sum'}).reset_index()


In [ ]:
categories = list(pvt4.ProfCat)

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=list(pvt4.DataPerson),
      theta=categories,
      fill='toself',
      name='DataPerson'
))
fig.add_trace(go.Scatterpolar(
      r=list(pvt4.Other),
      theta=categories,
      fill='toself',
      name='Other'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=False,
      range=[0, 0.5]
    )),
  showlegend=True
)

fig.show()

 # Answer 3
 
 - It would seem that Data People have more Master's and Doctorate degrees than the programmer counterparts. 

#  Conclusion

 - in conclusion, it rocks to be a datascientist .
 - but for real, it's a great career, 

# Question 4
- Experience levels

In [29]:
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software is about the same,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,"Taught yourself a new language, framework, or tool without taking a formal course",NaN,NaN,4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows,Django;Flask,Flask;jQuery,Node.js,Node.js,IntelliJ;Notepad++;PyCharm,Windows,I do not use containers,NaN,NaN,Yes,"Fortunately, someone else has that title",Yes,Twitter,Online,Username,2017,A few times per month or weekly,Find answers to specific questions;Learn how to do things I didn’t necessarily look for,3-5 times per week,Stack Overflow was much faster,31-60 minutes,No,NaN,"No, I didn't know that Stack Overflow had a job board","No, and I don't know what those are",Neutral,Just as welcome now as I felt last year,Tech articles written by other developers;Industry news about technologies you're interested in;Courses on technologies you're interested in,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software is about the same,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",NaN,Taken an online course in programming or software development (e.g. a MOOC),NaN,"Developer, desktop or enterprise applications;Developer, front-end",NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,I am actively looking for a job,I've never had a job,NaN,NaN,"Financial performance or funding status of the company or organization;Specific department or team I'd be working on;Languages, frameworks, and other technologies I'd be working with","Something else changed (education, award, media, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C++;HTML/CSS;Python,C++;HTML/CSS;JavaScript;SQL,NaN,MySQL,Windows,Windows,Django,Django,NaN,NaN,Atom;PyCharm,Windows,I do not use containers,NaN,Useful across many domains and could change many aspects of our lives,Yes,Yes,Yes,Instagram,Online,Username,2017,Daily or almost daily,Find answers to specific questions;Learn how to do things I didn’t necessarily look for,3-5 times per week,Stack Overflow was much faster,11-30 minutes,Yes,A few times per month or weekly,"No, I knew that Stack Overflow had a job board but have never used or visited it","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,Tech articles written by other developers;Industry news about technologies you're interested in;Tech meetups or events in your area;Courses on technologies you're interested in,19.0,Man,No,Straight / Heterosexual,NaN,No,

In [34]:
(set(df1.Employment))

{'Employed full-time',
 'Employed part-time',
 'Independent contractor, freelancer, or self-employed',
 'Not employed, and not looking for work',
 'Not employed, but looking for work',
 'Retired',
 nan}